In [2]:
!pip install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [1]:
import mod_sql as ms
import pandas as pd

In [2]:
## 대면
sql_class = ms.Database("172.16.12.149", 'ubion', "1234")
## 비대면면
sql_class_2 = ms.Database("darkpreist.iptime.org", "ubion", "1234")

1. emp라는 변수에 sql에 있는 emp table의 전체 데이터를 데이터프레임으로 저장
2. dept라는 변수에 sql에 있는 dept table의 전체 데이터를 데이터프레임으로 저장

In [3]:
sql = """
    select 
    * 
    from 
    emp
"""
emp = sql_class.executeAll(sql)
emp.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0
1,7499.0,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0
2,7521.0,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
4,7654.0,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0


In [4]:
sql = """
    select 
    * 
    from 
    dept
"""
dept = sql_class.executeAll(sql)
dept

,DEPTNO,DNAME,LOC
0,10.0,ACCOUNTING,NEW YORK
1,20.0,RESEARCH,DALLAS
2,30.0,SALES,CHICAGO
3,40.0,OPERATIONS,BOSTON


## pandas 복습
1. 새로운 파생변수 생성 job_code
    - salesman은 1, manager는 2, 그 외의 직업은 3
    - 컬럼은 EMPNO, ENAME, job_code만 출력

In [11]:
emp['JOB']

## apply, def 
def change(x):
    if x == 'SALESMAN':
        return 1
    elif x == "MANAGER":
        return 2
    else :
        return 3
emp['JOB'].apply(change)

## apply, lambda

emp['JOB'].apply(lambda x : 1 if (x == 'SALESMAN') else 
    (2 if (x == 'MANAGER') else 3))

emp2 = emp.copy()

## for문
for i in emp2.index:
    if emp2.loc[i, 'JOB'] == 'SALESMAN':
        emp2.loc[i, 'job_code'] = 1
    elif emp2.loc[i, 'JOB'] == 'MANAGER':
        emp2.loc[i, 'job_code'] = 2
    else :
        emp2.loc[i, 'job_code'] = 3
emp2[["EMPNO", "ENAME", "job_code"]]


,EMPNO,ENAME,job_code
0,7369.0,SMITH,3.0
1,7499.0,ALLEN,1.0
2,7521.0,WARD,1.0
3,7566.0,JONES,2.0
4,7654.0,MARTIN,1.0
5,7698.0,BLAKE,2.0
6,7782.0,CLARK,2.0
7,7788.0,SCOTT,3.0
8,7839.0,KING,3.0
9,7844.0,TURNER,1.0


In [12]:
## 위의 결과를 sql 쿼리문을 이용하여 생성
sql = """
    select 
    EMPNO, 
    ENAME, 
    JOB, 
    CASE 
        when JOB = 'SALESMAN'
        then 1 
        when JOB = 'MANAGER' 
        then 2
        else 3
    END as job_code 
    from 
    EMP
"""
sql_class.executeAll(sql)

,EMPNO,ENAME,JOB,job_code
0,7369.0,SMITH,CLERK,3
1,7499.0,ALLEN,SALESMAN,1
2,7521.0,WARD,SALESMAN,1
3,7566.0,JONES,MANAGER,2
4,7654.0,MARTIN,SALESMAN,1
5,7698.0,BLAKE,MANAGER,2
6,7782.0,CLARK,MANAGER,2
7,7788.0,SCOTT,ANALYST,3
8,7839.0,KING,PRESIDENT,3
9,7844.0,TURNER,SALESMAN,1


### pandas 복습 문제
1. emp에서 SAL 컬럼의 값이 2000 이상이고 5000 미만인 사원 정보를 출력

In [18]:
emp[(emp['SAL'] >= 2000) & (emp['SAL'] < 5000)]

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
3,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
5,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
6,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
7,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
12,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0


In [19]:
sql = """
    select 
    * 
    from 
    emp 
    where 
    SAL >= 2000 
    and 
    SAL < 5000
"""
sql_class.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
1,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
2,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
3,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
4,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0


In [20]:
sql = """
    select
    * 
    from 
    emp 
    where 
    SAL 
    between 
    2000 and 4999
"""
sql_class.executeAll(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0
1,7698.0,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0
2,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0
3,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0
4,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0
